In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator( rescale = 1.0/255,
                                   validation_split = 0.01)

In [ ]:
train_generator = datagen.flow_from_directory( directory = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/',
                                                     target_size = (224, 224),
                                                     class_mode = 'categorical',
                                                     batch_size = 128,
                                                     shuffle = True,
                                                     subset = 'training')


In [ ]:
validation_generator_2 = datagen.flow_from_directory( directory = '../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/',
                                                     target_size = (224, 224),
                                                     class_mode = 'categorical',
                                                     batch_size = 29)


In [ ]:
validation_generator = datagen.flow_from_directory( directory = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/',
                                                     target_size = (224, 224),
                                                     class_mode = 'categorical',
                                                     batch_size = 128,
                                                     subset = 'validation')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape = (224, 224, 3))

In [ ]:
base_model.summary()

In [ ]:
len(base_model.layers)

In [ ]:
i = 0
for layers in base_model.layers:
  layers.trainable = False
  i = i + 1
  if(i > 19):
    break


In [ ]:
x=base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.4)(x)
preds = Dense(29,activation='softmax')(x)
model = Model(inputs=base_model.input,outputs=preds)

In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch = 86130//128, epochs = 12, validation_data = validation_generator, validation_steps = 870//128)

** Total 12 epochs and 20 freeze layers of MobileNet_v2

In [ ]:
scoreSeg = model.evaluate_generator(validation_generator, 870//128)
print("Accuracy = ", scoreSeg[1])
print(scoreSeg)

In [ ]:
model.save("main_model_12e.h5")

In [ ]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

---------------------- Thank You -------------------------------